# TODO even the contribution

# Load Data

In [ ]:
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt
df = pandas.read_csv('d.csv', index_col=0)
df = df.filter(like='ETF', axis=0)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns = range(df.shape[1] - 1)
r = (b - a) / a
#p = 1 / np.linalg.norm(r, axis=1)
p = ( 1 / np.abs(r).mean(axis=1)).values
print(p, p.shape)
r = (r.T * p).T.values
l = r.shape[0]

cor = np.ones([l, l])
for i in range(l):
    for j in range(i + 1, l):
        s = (r[i] * r[j]).sum()
        cor[i][j] = s
        cor[j][i] = s
        
d = df.values.astype(np.float32)
p = p / d.T[0]
d = (d.T * p).T
print(d.shape)

In [ ]:
a = d[:, 1:]
b = d[:, :-1]
r = a - b
contri= np.abs(r).mean(axis=1)
print(contri, contri.shape)
contri = contri / contri.min()
print(contri)
print(contri.shape)

# Test random output distribution

In [ ]:
import random

dist = []
n = 6
for i in range(100000):
    choice = random.sample(range(d.shape[0]), n)
    # if 372 not in choice:
    #     choice = choice + [372]
    bad = True
    while bad:
        bad = False
        for k in range(len(choice)):
            for l in range(k):
                if np.abs(cor[choice[k]][choice[l]]) > 0.75:
                    bad = True
                    break
            if bad:
                break
        if bad:
            choice = random.sample(range(d.shape[0]), n)
            # if 372 not in choice:
            #     choice = choice + [372]

    s = d[choice].sum(axis=0)
    s = s / n
    m = s.mean()
    s = s / m
    x = np.arange(len(s)) + 1
    a, b = np.polyfit(x, np.log(s), 1)
    y = np.exp(a * x + b)

    # reg times
    last = s[0] < y[0]
    reg = 0
    regDist = [0]
    for i in range(1, len(s)):
        if (s[i] < y[i]) != last:
            regDist.append(i)
            reg += 1
            last = s[i] < y[i]

    # reg distribution
    regDist = np.array(regDist)
    regStd = (regDist[1:] - regDist[:-1]).std()
    dist.append(regStd)
    
pandas.DataFrame(dist).hist(bins=1000)
pandas.qcut(dist,  [0, 0.05, 0.1, 0.25, 0.5, 0.75, 1])

# Load Weekly data and clean into d.csv

In [ ]:
%matplotlib inline
import pandas
from glob import glob
import numpy as np

df = None
t = 'day'
for f in glob('D:\\work\\main\\IB\\*_' + t + '.csv'):
    if df is None:
        df = pandas.read_csv(f, index_col=0).T
    else:
        df = pandas.concat([df,pandas.read_csv(f, index_col=0).T], axis=1)


### make sure no Nan value is in d.csv

In [ ]:
if t == 'week':
    d = df.iloc[-112:-1,:]
    d = d.dropna(axis=1,how='any').astype('float32').T
else:
    d = df.T
    
if t == 'week':
    d.to_csv('d.csv')

In [ ]:
got = ['iShares Edge MSCI Min Vol EAFE ETF', '台湾ETF-iShares MSCI', '银行指数ETF-SPDR KBW', '道指三倍做空ETF-ProShares']
(df[got] * p[idx]).plot()

dates = df.index.astype(np.float32)
dates = np.round((dates - startDate) / 3600 / 24 / 1000) / 7 + 1

y = np.exp(a * dates + b)
y2 = (df[got] * p[idx]).sum(axis=1) / len(got)
plt.figure()
plt.plot(dates, y)
plt.plot(dates[30:], y2[30:])
y2[-1] / y[-1] - 1

In [ ]:
idx = [21, 36, 54, 68, 92]

columns = list(np.array(df.index)[idx])
print(columns)

coridx = np.ones([len(idx), len(idx)])
for i in range(len(idx)):
    for j in range(i):
        if np.abs(cor[idx[i]][idx[j]]) > 0.75:
            print('correlation too high')
        coridx[i][j] = cor[idx[i]][idx[j]]
        coridx[j][i] = cor[idx[j]][idx[i]]
print(coridx)

a = d[idx][:, 1:]
b = d[idx][:, :-1]
r = a - b
contri= np.abs(r).mean(axis=1)
contri = contri / contri.min()
print(contri)

graph = pandas.DataFrame(d[idx]).T
graph.columns = columns 
graph.plot()
d[idx].sum(axis=0)[-1]



s = (d[idx]).sum(axis=0)
print(s.shape)
print(p[idx])

dates = df.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1

s = s / len(idx)
a, b = np.polyfit(dates, np.log(s), 1)
print('a,b', a, b)
y = np.exp(a * dates + b)
last = s[0] < y[0]
reg = 0
for i in range(1, len(s)):
    if (s[i] < y[i]) != last:
        reg += 1
        last = s[i] < y[i]
print(reg)

plt.figure()
plt.plot(dates, y)
plt.plot(dates, s)